In [5]:
search_for = 121
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.003934144973754883
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 15
all cases: 237758115
type: [1, 1, 1, 1, 121] 121
cases of this type: 214358881
100000 7.575083522433671
200000 15.127716128029407
300000 19.078090278985528
400000 17.546967715710572
500000 18.4095623337815
600000 18.02011992264516
700000 19.645340379912724
800000 16.649025601307432
900000 11.045991899017464
1000000 14.687251795507326
1100000 16.96929121571228
1200000 17.04817619416988
1300000 17.921109005422714
1400000 17.919569257532093
1500000 17.261547980497816
1600000 17.293911704679754
1700000 17.49156996588093
1800000 8.665866156408995
1900000 15.694654595640497
2000000 21.476512075596602
2100000 21.30758460553807
2200000 20.204518192220284
2300000 20.940317219863235
2400000 21.89854577810566
2500000 19.14402601487869
2600000 21.013733523572586
2700000 7.6015566884546875
2800000 19.5113319711965
2900000 20.618541807177508
3000000 20.

29500000 13.482686535214986
29600000 14.910225660676511
29700000 14.350108828977596
29800000 15.907654033411099
29900000 8.693901286913889
30000000 15.048872006955296
30100000 15.667781913238557
30200000 15.31771814002839
30300000 15.520272619568377
30400000 8.56995841025412
30500000 14.903676809493954
30600000 15.285702006095837
30700000 15.35454978600396
30800000 14.877503933921545
30900000 14.013769477331042
31000000 10.751611508228264
31100000 9.831108264026813
31200000 13.891964166259815
31300000 15.011302260720685
31400000 14.743725691191665
31500000 15.79883721812879
31600000 16.045306308930446
31700000 8.702763829259975
31800000 15.438960480615139
31900000 15.252791211918797
32000000 15.794228376913338
32100000 15.073832322193889
32200000 7.456270897569527
32300000 16.236206776708833
32400000 13.769270457493018
32500000 13.916343865010163
32600000 13.718100173133037
32700000 13.305666434322148
32800000 6.897998464961738
32900000 13.006342416703152
33000000 12.942746603457948
33

59100000 11.90852032089172
59200000 11.515446106616908
59300000 12.022398098830676
59400000 5.404078068362723
59500000 12.16371801445975
59600000 11.905350276046732
59700000 11.762081698885472
59800000 6.639560627820289
59900000 13.24238230434075
60000000 12.70021074243024
60100000 13.595532227234537
60200000 13.565470652136385
60300000 13.522443009093534
60400000 13.794227933435224
60500000 12.979901121470371
60600000 12.31381689217651
60700000 12.618376908162476
60800000 5.750170966869185
60900000 12.397218379976788
61000000 12.77792900143832
61100000 10.40040532739431
61200000 6.715063546506296
61300000 11.49854185877757
61400000 12.196304788662996
61500000 8.37017314634618
61600000 9.15140982671397
61700000 12.107803519212974
61800000 12.898103904273748
61900000 13.155670419295515
62000000 12.655620510049085
62100000 13.157436249082325
62200000 13.817789802529193
62300000 12.805877817968186
62400000 12.719462597555637
62500000 11.372019118843912
62600000 6.474020683337798
62700000 

88900000 10.154089637859027
89000000 9.54624117318309
89100000 10.28850529594196
89200000 10.083969397339304
89300000 8.807252352241472
89400000 6.501507665149927
89500000 4.046698281070845
89600000 8.9985023084424
89700000 8.015506613470654
89800000 9.800715856247491
89900000 9.83687737639957
90000000 10.525085056510209
90100000 10.120443331737578
90200000 9.707026561495901
90300000 11.478167366372228
90400000 11.424104518735323
90500000 10.585420851974057
90600000 10.817254233685007
90700000 9.989006636752487
90800000 9.962130169432534
90900000 10.161401280303856
91000000 10.113400585090098
91100000 7.560031122690595
91200000 7.744458533881158
91300000 4.2808012378008
91400000 5.396075315790093
91500000 10.521507770879616
91600000 10.353502433451037
91700000 9.800973261315342
91800000 10.094446001027276
91900000 10.09224805296408
92000000 10.456980450498571
92100000 10.5823446201185
92200000 11.243022481934569
92300000 10.468129893097824
92400000 9.834599951155484
92500000 10.5051651

118200000 7.858324508987337
118300000 7.935319011262427
118400000 7.43300176090764
118500000 8.121192676886011
118600000 8.045837795357022
118700000 9.006099353525993
118800000 8.646301517130823
118900000 8.158363584977389
119000000 7.981511361818694
119100000 7.847509427591621
119200000 7.893223375179562
119300000 7.904280088288447
119400000 7.924846092756808
119500000 5.151415467031652
119600000 3.6939589677237503
119700000 4.055129613537709
119800000 7.195139341698318
119900000 7.966753896198707
120000000 7.840991900875896
120100000 8.030627390758257
120200000 7.5456561927640475
120300000 7.7423847216879125
120400000 8.363427835650938
120500000 8.735276572585768
120600000 8.892899813042293
120700000 7.786640941245689
120800000 7.355574915460887
120900000 7.691815726617426
121000000 7.5944219943384725
121100000 7.380376470484091
121200000 7.307225814043384
121300000 4.54669370747198
121400000 3.598343155162437
121500000 4.7670475289675
121600000 7.20373170544871
121700000 7.859469607

147400000 6.900910109011179
147500000 6.273413426556617
147600000 4.128130215951502
147700000 5.560099245734247
147800000 6.2529813902003735
147900000 5.516596055644989
148000000 3.1057467180299194
148100000 6.22180847894912
148200000 5.902659784864177
148300000 3.3407892013161216
148400000 6.356107955499285
148500000 6.311924450532774
148600000 6.934025654301369
148700000 7.031517291225089
148800000 6.642285365202427
148900000 6.63549142303493
149000000 7.182510423353066
149100000 6.386325967856845
149200000 6.527019419395884
149300000 6.0051087390250295
149400000 3.345315020351787
149500000 5.825751437547903
149600000 5.923632525512666
149700000 4.079796702652366
149800000 4.557012977892716
149900000 6.067418096398182
150000000 5.928452880418828
150100000 3.068993338768234
150200000 5.755581113261815
150300000 6.188503064775963
150400000 6.483317177824281
150500000 6.693338584085938
150600000 6.455216328029394
150700000 6.513990104590661
150800000 6.40972871590912
150900000 6.3169652

176600000 4.1705905271078905
176700000 4.346582739580154
176800000 4.716343722637203
176900000 2.299417930630356
177000000 4.629102040321707
177100000 4.652086984161103
177200000 4.700513726053738
177300000 4.269646076684763
177400000 4.63570845678241
177500000 2.5094453110475707
177600000 4.453955855081945
177700000 4.417461248179409
177800000 3.9673542760537006
177900000 4.222187887968213
178000000 4.14285658401062
178100000 1.8079491008109088
178200000 3.9151153878446023
178300000 4.212538228941229
178400000 4.503722057671958
178500000 4.387045906816811
178600000 4.25081160285497
178700000 2.858755357717064
178800000 4.64610836532718
178900000 4.656950273881899
179000000 4.721056646693879
179100000 4.1673136478827
179200000 3.7164291928199793
179300000 3.224773006290075
179400000 4.188087300098002
179500000 4.029268963652449
179600000 4.375949806687783
179700000 3.890561261871795
179800000 3.1790542810972493
179900000 2.4306299112397762
180000000 3.956724472976645
180100000 3.720011

205500000 1.5335604140641985
205600000 1.3312806668511463
205700000 2.522635119737347
205800000 2.4521853926602404
205900000 2.2851729609111646
206000000 2.3808069965759384
206100000 2.3019924390797515
206200000 2.195383161322127
206300000 2.2713351427390807
206400000 1.2620979845869542
206500000 2.0768081894883843
206600000 2.269061290223963
206700000 2.2445252535020512
206800000 2.281835515405168
206900000 2.4856765382135944
207000000 2.3265817553893524
207100000 2.3020172395679817
207200000 1.8741083448340066
207300000 1.4528957512316407
207400000 1.500934653684149
207500000 2.2928481449529428
207600000 2.287461984336883
207700000 2.4454934506628145
207800000 2.213933812096427
207900000 2.303036014998615
208000000 2.1605617458179593
208100000 2.2229440033949115
208200000 0.8969809190149606
208300000 2.2952087089544766
208400000 2.2681745667988986
208500000 2.176744184948246
208600000 2.180885529967533
208700000 2.214229280420426
208800000 2.184570320687175
208900000 2.16360064409641

233400000 0.13554571323549747
233500000 0.12833207035545838
233600000 0.14059622918009426
233700000 0.12124393521788715
233800000 0.12121385729669532
best so far: 0
type: [1, 1, 1, 121, 1] 121
cases of this type: 1771561
233900000 0.07469471317115757
234000000 0.024115491546114285
234100000 0.023278423859811492
234200000 0.022764217657589247
234300000 0.022187878228455783
234400000 0.02143132240542438
234500000 0.020873793254752953
234600000 0.020372012999574342
234700000 0.019855217227607965
234800000 0.019272437376807134
234900000 0.01858432448889812
235000000 0.01723414537092712
235100000 0.016652377087685796
235200000 0.016087783580750227
235300000 0.01526236917047037
235400000 0.015558575083563723
235500000 0.014385261926184098
235600000 0.01408414713619815
best so far: 0
type: [1, 1, 11, 1, 11] 121
cases of this type: 1771561
235700000 0.054396507354819115
235800000 0.05746431950941682
235900000 0.04890519761548771
236000000 0.04750499284502865
236100000 0.04652940687775612
23620